In [1]:
!curl -XGET "http://128.230.247.186:9201/_cat/indices" | grep kimun_test

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1300  100  1300    0     0  44339      0 --:--:-- --:--:-- --:--:-- 44827
yellow open kimun_test                  PYCHVMihRsyJBYxNqu-pBQ 5 1    99932       0     1gb     1gb


In [2]:
!curl -XDELETE "http://128.230.247.186:9201/kimun_test"
!curl -XPUT "http://128.230.247.186:9201/kimun_test" -H 'Content-Type: application/json' -d '{"settings": {"analysis": {"filter": {"alphabets_char_filter": {"pattern": "[^a-zA-Z]", "type": "pattern_replace", "replacement": ""}, "word_joiner": {"catenate_all": "true", "type": "word_delimiter"}, "autocomplete_flter": {"type": "edge_ngram", "min_gram": "1", "max_gram": "150"}, "edge_ngram": {"token_chars": ["letter", "digit"], "min_gram": "1", "type": "edgeNGram", "max_gram": "50"}, "english_poss_stemmer": {"name": "possessive_english", "type": "stemmer"}, "filter_stop": {"type": "stop", "stopwords": "_english_"}, "filter_shingle": {"type": "shingle", "max_shingle_size": 4, "min_shingle_size": 2, "output_unigrams": "true"}}, "analyzer": {"keyword_analyzer": {"filter": ["lowercase", "alphabets_char_filter", "trim", "english_poss_stemmer"], "tokenizer": "standard"}, "autocomplete": {"filter": ["lowercase", "word_joiner", "autocomplete_flter"], "type": "custom", "tokenizer": "my_tokenizer"}, "edge_ngram_analyzer": {"filter": ["lowercase", "alphabets_char_filter", "trim", "english_poss_stemmer", "edge_ngram"], "tokenizer": "standard"}, "analyzer_shingle": {"tokenizer": "standard", "filter": ["standard", "lowercase", "stop", "filter_shingle"]}, "english_analyzer": {"type": "standard", "stopwords": "_english_"}}, "tokenizer": {"my_tokenizer": {"pattern": ";", "type": "pattern"}}}}, "mappings": {"documents": {"properties": {"city": {"type": "text", "fields": {"keyword": {"type": "keyword", "ignore_above": 256}}}, "country": {"type": "text", "fields": {"keyword": {"type": "keyword", "ignore_above": 256}}}, "date": {"type": "text", "fields": {"keyword": {"type": "keyword", "ignore_above": 256}}}, "documentType": {"type": "text", "fields": {"keyword": {"type": "keyword", "ignore_above": 256}}}, "endDate": {"type": "text", "fields": {"keyword": {"type": "keyword", "ignore_above": 256}}}, "id": {"type": "text", "fields": {"keyword": {"type": "keyword", "ignore_above": 256}}}, "organizations": {"type": "text", "fields": {"keyword": {"type": "keyword", "ignore_above": 256}}}, "otherID": {"type": "text", "fields": {"keyword": {"type": "keyword", "ignore_above": 256}}}, "scientists": {"type": "text", "fields": {"keyword": {"type": "keyword", "ignore_above": 256}, "scientists_search": {"type": "text", "analyzer": "edge_ngram_analyzer", "search_analyzer": "keyword_analyzer"}}, "analyzer": "edge_ngram_analyzer", "search_analyzer": "keyword_analyzer"}, "source": {"type": "text", "fields": {"keyword": {"type": "keyword", "ignore_above": 256}}}, "sourceID": {"type": "text", "fields": {"keyword": {"type": "keyword", "ignore_above": 256}}}, "summary": {"type": "text", "fields": {"keyword": {"type": "keyword", "ignore_above": 256}, "summary_search": {"type": "text", "analyzer": "analyzer_shingle"}, "summary_text": {"type": "text", "analyzer": "english_analyzer"}}}, "text": {"type": "text", "fields": {"keyword": {"type": "keyword", "ignore_above": 256}}, "analyzer": "english_analyzer"}, "title": {"type": "text", "fields": {"keyword": {"type": "keyword", "ignore_above": 256}, "title_search": {"type": "text", "analyzer": "analyzer_shingle"}, "title_text": {"type": "text", "analyzer": "english_analyzer"}}}, "topicNorm": {"type": "float"}, "venue": {"type": "text", "fields": {"keyword": {"type": "keyword", "ignore_above": 256}}}}}}}'

{"acknowledged":true}{"acknowledged":true,"shards_acknowledged":true,"index":"kimun_test"}

In [3]:
!curl -XGET "http://128.230.247.186:9201/_cat/indices" | grep kimun_test

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1300  100  1300    0     0  35428      0 --:--:-- --:--:-- --:--:-- 36111
yellow open kimun_test                  -Fmr6c_YQsSx4RL6XilcaQ 5 1        0       0   1.1kb   1.1kb


In [4]:
# i'm loading a sample that i created from the latest topic_svd
# the sample is saved here: '/user/hzhan212/eileen/svd.sample'

# and i'm testing on the index "kimun_test"

In [5]:
#!/usr/bin/env python
# coding: utf-8

import os, findspark

# temporarily changing PYSPARK_PYTHON to avoid rdd error.
os.environ['PYSPARK_PYTHON'] = '/home/tozeng/anaconda3/bin/python'
os.environ['PYSPARK_DRIVER_PYTHON'] = '/home/tozeng/anaconda3/bin/python'

findspark.init('/opt/cloudera/parcels/SPARK2-2.4.0.cloudera2-1.cdh5.13.3.p0.1041012/lib/spark2/')

import pyspark
import scipy.sparse, json
import numpy as np
from pyspark import SparkContext
from pyspark.sql import functions, SparkSession, SQLContext
from pyspark.sql import functions as F
from pyspark.sql.functions import udf, col, unix_timestamp, from_unixtime, coalesce, to_date
from pyspark.ml.linalg import Vectors, _convert_to_vector, VectorUDT


def create_session():
    spark = SparkSession.builder.\
        config("spark.executor.instances", '5').\
        config("spark.executor.memory", '40g').\
        config('spark.executor.cores', '9').\
        config('spark.cores.max', '9').\
        config("spark.kryoserializer.buffer.max.mb", "2000").\
        config('spark.jars', '/home/eileen/nsf_data_ingestion/libraries/elasticsearch-hadoop-7.10.1.jar').\
        appName('kimun_loader').\
        getOrCreate()
    return spark

def parse(tup):
    d = {}
    d['id']=tup['id']
    d['city']=tup['city']
    d['country'] = tup['country']
    d['date'] = tup['date']
    d['documentType'] = tup['type']
    d['endDate'] = tup['end_date']
    d['organizations'] = tup['organizations']
    d['otherID'] = tup['other_id']
    d['scientists'] = tup['scientists']
    d['sourceID'] = tup['source_id']
    d['source'] = tup['source']
    d['summary'] = tup['abstract']
    d['text'] = tup['content']
    d['title'] = tup['title']
    d['venue']=tup['venue']
    d['topicNorm'] = list(tup['topic'])
    return (d['id'], json.dumps(d))


def dense_to_sparse(vector):
    sparse = _convert_to_vector(scipy.sparse.csc_matrix(vector.toArray()).T)
    return sparse

def to_date_(col, formats=("MM/dd/yyyy", "yyyy")):
    return coalesce(*[to_date(col, f) for f in formats])

def kimun_load():
    spark = create_session()
    sqlContext = SQLContext(spark.sparkContext)  
   
    topic_df = sqlContext.read.parquet('/user/hzhan212/eileen/svd.sample')

    #new date formatting
    topic_df.date = topic_df.select('date', from_unixtime(unix_timestamp('date', 'yyy')).alias('date'))
    split_col = pyspark.sql.functions.split(topic_df['date'], '-')
    topic_df = topic_df.withColumn('date', split_col.getItem(0))
    topic_df = topic_df.withColumn("formatted_date", to_date_("date"))
    topic_df.formatted_date = topic_df.select('formatted_date',from_unixtime(unix_timestamp('formatted_date','yyy')).alias('formatted_date'))
    split_col = pyspark.sql.functions.split(topic_df['formatted_date'], '-')
    topic_df = topic_df.withColumn('formatted_date', split_col.getItem(0))
    topic_df.date = topic_df.select(topic_df.formatted_date).alias('date')
    columns = topic_df.columns
    topic_df = topic_df.drop('date')
    topic_df = topic_df.withColumnRenamed('formatted_date', 'date')
    topic_df = topic_df.withColumn("title", F.regexp_replace(F.regexp_replace(F.regexp_replace("title", "\\]\\[", ""), "\\[",""),"\\]",""))
    topic_df = topic_df.withColumn("abstract", F.regexp_replace(F.regexp_replace(F.regexp_replace("abstract", "\\]\\[", ""), "\\[", ""), "\\]", ""))
    columns = topic_df.columns
    val2 = topic_df.select(columns).groupBy(['title', 'scientists', 'venue']).agg(F.min('date'))
    val2 = val2.withColumnRenamed('min(date)', 'date')
    val3 = val2.join(topic_df, ['title', 'scientists', 'venue', 'date'])
    val3 = val3.withColumn("date", val3["date"].cast("int"))
    val3 = val3.withColumn("date", val3["date"].cast("string"))
    #date formatting end
    sc = spark.sparkContext
    print("before conf ")
    es_write_conf = {
            "es.nodes" : "128.230.247.186",\
            "es.port" : "9201",\
            # TODO: check if the name of index is consistent with the current one
            "es.resource" : 'kimun_test/documents',\
            "es.input.json": "yes",\
            "es.mapping.id": "id",\
            "es.batch.size.entries": "5000",\
            "es.batch.write.retry.wait": "3000"
        }

    rdd = sc.newAPIHadoopRDD("org.elasticsearch.hadoop.mr.EsInputFormat", "org.apache.hadoop.io.NullWritable", "org.elasticsearch.hadoop.mr.LinkedMapWritable", conf=es_write_conf)
    topic_rdd = val3.rdd
    result = topic_rdd.map(parse)
    result = result.repartition(1)
    print("after repartition")

    result.saveAsNewAPIHadoopFile(
        path='-',
        outputFormatClass="org.elasticsearch.hadoop.mr.EsOutputFormat",
        keyClass="org.apache.hadoop.io.NullWritable",
        valueClass="org.elasticsearch.hadoop.mr.LinkedMapWritable",
        conf=es_write_conf)

    spark.stop()

In [6]:
kimun_load()

before conf 
after repartition


In [7]:
!curl -XGET "http://128.230.247.186:9201/_cat/indices" | grep kimun_test

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1300  100  1300    0     0  51062      0 --:--:-- --:--:-- --:--:-- 52000
yellow open kimun_test                  -Fmr6c_YQsSx4RL6XilcaQ 5 1    99932       0     1gb     1gb


In [3]:
!curl -XGET "http://128.230.247.186:9201/kimun_test/_search?q=computer"

{"took":2229,"timed_out":false,"_shards":{"total":5,"successful":5,"skipped":0,"failed":0},"hits":{"total":814,"max_score":10.860512,"hits":[{"_index":"kimun_test","_type":"documents","_id":"medline_16428048","_score":10.860512,"_source":{"id": "medline_16428048", "city": null, "country": null, "date": "2006", "documentType": "publication", "endDate": null, "organizations": "Department of Radiology, The University of Chicago, Chicago, IL 60637, USA. y-jiang@uchicago.edu", "otherID": null, "scientists": "Y Jiang; CE Metz", "sourceID": "16428048", "source": "medline", "summary": "RATIONALE AND OBJECTIVES\nSome computer-aided diagnosis (CAD) methods produce a quantitative diagnostic assessment (eg, likelihood of malignancy) based on computer image analysis that a radiologist who uses the computer aid must combine with his or her own assessment. Observer studies show that although CAD helps radiologists improve diagnostic performance, ad hoc use of computer aid can produce performance infe

In [1]:
!curl -XGET "http://128.230.247.186:9201/_cat/indices" | grep kimun

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1300  100  1300    0     0  26754      0 --:--:-- --:--:-- --:--:-- 27083
yellow open kimun                       TumBDdASQ0qY3hu7yM2pIg 5 1 26911345      58 239.9gb 239.9gb
yellow open kimun_version4              Bxq_pJiqTNOF0UEgD2SFew 5 1 25871670 5749145 276.6gb 276.6gb
yellow open kimun_test                  -Fmr6c_YQsSx4RL6XilcaQ 5 1    99932       0     1gb     1gb


In [2]:
!curl -XGET "http://128.230.247.186:9201/kimun/_search?q=computer"

{"took":14761,"timed_out":false,"_shards":{"total":5,"successful":5,"skipped":0,"failed":0},"hits":{"total":249446,"max_score":11.7774315,"hits":[{"_index":"kimun","_type":"documents","_id":"medline_10270073","_score":11.7774315,"_source":{"id": "medline_10270073", "city": null, "country": null, "date": "1985", "documentType": "publication", "endDate": null, "organizations": "", "otherID": null, "scientists": "D Winsten", "sourceID": "10270073", "source": "medline", "summary": "", "text": "  D Winsten ", "title": "Good computer--bad computer.", "venue": "", "topicNorm": [2.578656197829747e-06, 1.770699318869986e-06, 3.88726372563267e-06, -2.7749300682089424e-06, 1.0054390619020725e-05, -1.0169152828314832e-05, -1.9894296371014604e-06, 1.019801068954311e-06, -1.7003953370787927e-06, -8.181430190945428e-07, -4.968861757188824e-06, -6.8621917715325584e-06, -4.439384847339861e-06, -4.368701946818587e-06, -1.6842138138796881e-06, -3.033932694808896e-06, -3.5034044283796242e-06, -5.295473904